In [2]:
import numpy as np
import scipy.optimize as opt
import sympy as sp

In [15]:
# This application must solve numerically the following two-dimensional partial differential equation:
# -div(K*grad(u)) + v*grad(u) = f
# where K is a two-by-two matrix function of x and y, v is a vector function of x and y, and f is a scalar function of x and y.

In [6]:
# Define step size
h = 0.01

# Define x boundaries
x_min = 0
x_max = 1

# Define y boundaries
y_min = 0
y_max = 1

# Define the number of nodes in x
N_x = int(1 / h)

# Define the number of nodes in y
N_y = int(1 / h)

# Define K - a two-by-two matrix function depending on x and y
K = lambda x, y : np.matrix([[np.sin(x), np.cos(x)], [np.cos(y), np.sin(y)]])

# Define v - a two dimensional vector function v depends on x and y
v = lambda x, y : np.array([np.cos(x), np.sin(y)])

# Define f - a scalar function depending on x and y
f = lambda x, y : np.sin(x) * np.cos(y)

# Define matrix u - solve the equation -div(K*grad(u)) + v*grad(u) = f
u = np.zeros((N_x, N_y))

# Define the set of neighbors of node i,j
def neighbors(i, j):
    if i == 0:
        if j == 0:
            return [(i, j + 1), (i + 1, j)]
        elif j == N_y - 1:
            return [(i, j - 1), (i + 1, j)]
        else:
            return [(i, j - 1), (i, j + 1), (i + 1, j)]
    elif i == N_x - 1:
        if j == 0:
            return [(i - 1, j), (i, j + 1)]
        elif j == N_y - 1:
            return [(i - 1, j), (i, j - 1)]
        else:
            return [(i - 1, j), (i, j - 1), (i, j + 1)]
    elif j == 0:
        return [(i - 1, j), (i + 1, j), (i, j + 1)]
    elif j == N_y - 1:
        return [(i - 1, j), (i + 1, j), (i, j - 1)]
    else:
        return [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
    
# Define function that return normal between two nodes
def normal(node_1, node_2):
    x_1, y_1 = tuple(h*np.array(node_1))
    x_2, y_2 = tuple(h*np.array(node_2))
    return np.array([y_2 - y_1, x_2 - x_1]) / np.sqrt((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2)

# Define function that return point between two nodes
def middle_point(node_1, node_2):
    x_1, y_1 = tuple(h*np.array(node_1))
    x_2, y_2 = tuple(h*np.array(node_2))
    return np.array([(x_1 + x_2) / 2, (y_1 + y_2) / 2])

# Define function that solves the system of nonlinear equations
def solve_nonlinear_equations(u):
    # Convert u into an N_x by N_y matrix
    u = np.reshape(u, (N_x, N_y))
    # Initialize equations
    equations = []
    # Fill equations
    for i in range(1, N_x - 1):
        for j in range(1, N_y - 1):
            node = (i, j)
            coefficient = 0
            for neighbor_node_1 in neighbors(i, j):
                first_coefficient = 0
                for neighbor_node_2 in neighbors(i, j):
                    middle_point_2 = middle_point(node, neighbor_node_2)
                    first_coefficient += K(middle_point_2[0], middle_point_2[1]).transpose().dot(normal(node, neighbor_node_2)).dot(normal(node, neighbor_node_1))
                middle_point_1 = middle_point(node, neighbor_node_1)
                second_coefficient = h*v(middle_point_1[0], middle_point_1[1]).dot(normal(node, neighbor_node_1))
                coefficient += (-first_coefficient + second_coefficient)/(2*h**2)
            equations.append(coefficient*(u[neighbor_node_1[0]][neighbor_node_1[1]] - u[i][j]) - f(i*h, j*h))
    for i in range(0, N_x):
        equations.append(u[i][0])
        equations.append(u[i][N_y - 1])
    for j in range(1, N_y-1):
        equations.append(u[0][j])
        equations.append(u[N_x - 1][j])
    # Return equations
    return equations

# Convert u from an N_x by N_y matrix to array
u = np.reshape(u, (N_x*N_y, 1))

# Solve the system of nonlinear equations
u = np.array(opt.fsolve(solve_nonlinear_equations, u))

c:\Users\stas2\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
